In [ ]:
import sys
sys.path.append('../implementation')
import numpy as np
import pandas as pd
import ast
from tqdm import tqdm
from util import lognormpdf
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sp
from wall_bias import Wall
import warnings
warnings.filterwarnings('ignore')
from util import flatten_list

In [ ]:
# Loading the underlying data and user interaction data
data_path = '../data/boardrooms/boardrooms_data.csv'
ui_data_path = '../data/boardrooms/boardrooms_combined_interactions.csv'
output_file_path = '../output/boardrooms/boardrooms_wall.pkl'

underlying_data = pd.read_csv(data_path)
# underlying_data['industry_code'] = pd.factorize(underlying_data['industry'])[0]
interaction_data = pd.read_csv(ui_data_path)
interaction_data['interaction_session'] = interaction_data.apply(lambda row: ast.literal_eval(row.interaction_session), axis=1)

ks = [1, 5, 10, 20, 50, 100]
d_attributes = ['industry']
c_attributes = ['mktcap', 'unrelated', 'female', 'age', 'tenure', 'medianpay']

underlying_data = underlying_data[d_attributes + flatten_list(c_attributes)].copy()

In [ ]:
# Not necessary to run if we already have results file for Wall
# Running Wall through all user interaction sessions and saving results in file
wall_results = pd.DataFrame()

for participant_index, row in interaction_data.iterrows():
    print(f'Processing user {row.user} task {row.task}')
    results = {'participant_id': row.user, 'task': row.task}
    wall = Wall(underlying_data, c_attributes, d_attributes)
    for i in tqdm(range(len(interaction_data.iloc[participant_index].interaction_session))):
        interaction = interaction_data.iloc[participant_index].interaction_session[i]
        interaction_d = pd.DataFrame(data=underlying_data.iloc[interaction].to_dict(), index=[i])
        wall.update(interaction_d)

    bias = wall.get_attribute_bias()
    for col in bias.columns:
        results[f'bias-{col}'] = bias[col].to_numpy()
    
    wall_results = wall_results.append(results, ignore_index=True)
    
wall_results.to_pickle(output_file_path)